<a href="https://colab.research.google.com/github/pari1jay/Sentiment-Analysis-of-Financial-Complaints-Using-NLP/blob/main/Customer_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect Colab with Google Drive

# Mining Consumer Voices: A Sentiment Analysis of Financial Complaints Using NLP

I have here worked on a sentiment analysis project using real-world complaint data from the Consumer Financial Protection Bureau (CFPB) available on kaggle. Through this project I aim to uncover insights from complaint narratives and classify customer sentiment using natural language processing (NLP) techniques.

The customer sentiment metric helps organizations to pinpoint customer opinions and feelings about their business, products or services. It allows companies to see if their audience has a positive, negative or neutral view of their experiences. It can also help customers choose companies best for a particular product.

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd # data manipulation
import matplotlib.pyplot as plt #plotting
from wordcloud import WordCloud, STOPWORDS #filter stopwords

     
## 1. Understand the Dataset Structure

In [ ]:
df= pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/consumer_complaints.csv')
df.head()

/tmp/ipython-input-3-483941321.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/consumer_complaints.csv')


,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [ ]:
cp= df["product"].value_counts()
cp

,count
product,
Mortgage,186475
Debt collection,101052
Credit reporting,91854
Credit card,66468
Bank account or service,62563
Consumer Loan,20990
Student loan,15839
Payday loan,3877
Money transfers,3812


In [ ]:
df.shape


(555957, 18)

In [ ]:
df.columns

Index(['date_received', 'product', 'sub_product', 'issue', 'sub_issue',
       'consumer_complaint_narrative', 'company_public_response', 'company',
       'state', 'zipcode', 'tags', 'consumer_consent_provided',
       'submitted_via', 'date_sent_to_company', 'company_response_to_consumer',
       'timely_response', 'consumer_disputed?', 'complaint_id'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555957 entries, 0 to 555956
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   date_received                 555957 non-null  object
 1   product                       555957 non-null  object
 2   sub_product                   397635 non-null  object
 3   issue                         555957 non-null  object
 4   sub_issue                     212622 non-null  object
 5   consumer_complaint_narrative  66806 non-null   object
 6   company_public_response       85124 non-null   object
 7   company                       555957 non-null  object
 8   state                         551070 non-null  object
 9   zipcode                       551452 non-null  object
 10  tags                          77959 non-null   object
 11  consumer_consent_provided     123458 non-null  object
 12  submitted_via                 555957 non-null  object
 13 

| Column                         | Type        | Use in Sentiment Analysis                                                                   |
| ------------------------------ | ----------- | ------------------------------------------------------------------------------------------- |
| `date_received`                | Date        | Useful for **time-series analysis** of complaint trends or sentiment over time              |
| `product`                      | Categorical | Good for **grouping sentiment** by broad categories (e.g., Mortgage, Credit reporting)      |
| `sub_product`                  | Categorical | Adds **granular breakdown** under `product`; useful for deeper sentiment segmentation       |
| `consumer_complaint_narrative` | Text        | **Primary text** field for sentiment modeling and NLP                                       |
| `company`                      | Categorical | Enables **company-wise sentiment analysis** (e.g., compare banks' consumer sentiment)       |
| `state`, `zipcode`             | Categorical | Useful for **geographic distribution** of sentiment or complaint frequency                  |
| `consumer_consent_provided`    | Categorical | May indicate **data quality or completeness bias**; not directly used in sentiment modeling |
| `company_response_to_consumer` | Categorical | Helpful for **correlating sentiment vs. company action**                                    |
| `timely_response`              | Yes/No      | Could correlate with sentiment; useful for **response quality analysis**                    |
| `consumer_disputed?`           | Yes/No      | Can be used as a **proxy target (weak label)** for supervised sentiment modeling            |
| `complaint_id`                 | Numeric     | Just a **unique identifier**; not used in modeling                                          |


In [ ]:
prod_subprod = df.groupby('product')['sub_product'].value_counts()
prod_subprod

product                  sub_product                           
Bank account or service  Checking account                          44369
                         Other bank product/service                10895
                         Savings account                            3996
                         (CD) Certificate of deposit                2839
                         Cashing a check without an account          464
Consumer Loan            Vehicle loan                              12168
                         Installment loan                           5279
                         Vehicle lease                              1639
                         Personal line of credit                    1516
                         Title loan                                  329
                         Pawn loan                                    59
Debt collection          Other (i.e. phone, health club, etc.)     29617
                         I do not know                             21634
                         Credit card                               20825
                         Medical                                   13297
                         Payday loan                                5807
                         Mortgage                                   3433
                         Auto                                       2523
                         Non-federal student loan                   2130
                         Federal student loan                       1786
Money transfers          International money transfer               2035
                         Domestic (US) money transfer               1777
Mortgage                 Other mortgage                            74319
                         Conventional fixed mortgage               57182
                         Conventional adjustable mortgage (ARM)    20941
                         FHA mortgage                              19152
                         Home equity loan or line of credit         8944
                         VA mortgage                                3735
                         Reverse mortgage                           1537
                         Second mortgage                             665
Other financial service  Check cashing                               156
                         Debt settlement                             142
                         Money order                                  71
                         Credit repair                                54
                         Travelerâs/Cashierâs checks              51
                         Foreign currency exchange                    43
                         Refund anticipation check                    40
Payday loan              Payday loan                                3877
Prepaid card             General purpose card                       1165
                         Payroll card                                313
                         Mobile wallet                               240
                         Gift or merchant card                       225
                         Government benefit payment card             225
                         ID prepaid card                             144
                         Other special purpose card                  123
                         Transit card                                 30
                         Electronic Benefit Transfer / EBT card        5
Student loan             Non-federal student loan                  15839
Name: count, dtype: int64

In [ ]:
company_count_distinct= df['company'].nunique()
company_count_distinct

3605

The data here belongs to different banks and I can perform multi-dimensionsal analysis like: Company-wise Sentiment Comparison-but there are 3605 companies and blindly comparing all of them would lead to noise, not insight.

#### Product vs Sentiment Within Each Bank - I could say which bank has good feedback for a product and worst feedback for a product, this can help customers choose company based on their product.


Given a product (like “Credit card”, “Mortgage”), I want to:

- Generate a short narrative describing what consumers are saying — for the top 3 companies with most positive sentiment.
- List the companies to avoid -> top 3 companies with most negative sentiment for that product.

##2. Filter and Clean the Data





2.1 new df with relevant columns :

"consumer_complaint_narrative", "product", "company", "state", "submitted_via", "company_response_to_consumer", "timely_response", "consumer_disputed?

In [ ]:
cols_to_keep = ["consumer_complaint_narrative", "product", "company", "state", "submitted_via", "company_response_to_consumer", "timely_response", "consumer_disputed?"]
df_narr = df[cols_to_keep].copy() # new df for nlp
df_narr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555957 entries, 0 to 555956
Data columns (total 8 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   consumer_complaint_narrative  66806 non-null   object
 1   product                       555957 non-null  object
 2   company                       555957 non-null  object
 3   state                         551070 non-null  object
 4   submitted_via                 555957 non-null  object
 5   company_response_to_consumer  555957 non-null  object
 6   timely_response               555957 non-null  object
 7   consumer_disputed?            555957 non-null  object
dtypes: object(8)
memory usage: 33.9+ MB


Out of the total 555,957 rows in your dataset, only 66,806 rows have a non-null consumer_complaint_narrative. That means:

🔹 Only ~12% of complaints contain free-text narratives suitable for NLP or true sentiment analysis.





2.2 create 2 data sets | Handle Missing Values :

- Drop rows or columns if missing data is too frequent or irrelevant
- For text fields, drop rows without text if text is mandatory (like complaint narratives)
- Remove Duplicates
- Check for and remove exact or near-duplicate rows that might bias analysis

In [ ]:
cols_to_keep1 = ["product", "company", "state", "submitted_via", "company_response_to_consumer", "timely_response", "consumer_disputed?"]
df_new = df[cols_to_keep1].copy() # new df for other analysis
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555957 entries, 0 to 555956
Data columns (total 7 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   product                       555957 non-null  object
 1   company                       555957 non-null  object
 2   state                         551070 non-null  object
 3   submitted_via                 555957 non-null  object
 4   company_response_to_consumer  555957 non-null  object
 5   timely_response               555957 non-null  object
 6   consumer_disputed?            555957 non-null  object
dtypes: object(7)
memory usage: 29.7+ MB


In [ ]:
df_new.dropna(inplace=True)
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 551070 entries, 0 to 553096
Data columns (total 7 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   product                       551070 non-null  object
 1   company                       551070 non-null  object
 2   state                         551070 non-null  object
 3   submitted_via                 551070 non-null  object
 4   company_response_to_consumer  551070 non-null  object
 5   timely_response               551070 non-null  object
 6   consumer_disputed?            551070 non-null  object
dtypes: object(7)
memory usage: 33.6+ MB


In [ ]:
df_narr.dropna(inplace=True)
df_narr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66620 entries, 190126 to 553096
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   consumer_complaint_narrative  66620 non-null  object
 1   product                       66620 non-null  object
 2   company                       66620 non-null  object
 3   state                         66620 non-null  object
 4   submitted_via                 66620 non-null  object
 5   company_response_to_consumer  66620 non-null  object
 6   timely_response               66620 non-null  object
 7   consumer_disputed?            66620 non-null  object
dtypes: object(8)
memory usage: 4.6+ MB


##3. We perform text preprocessing to prepare the text data for the model building. It is the very first step of NLP projects. Some of the preprocessing steps are:

- Removing punctuations like . , ! $( ) * % @
- Removing URLs
- Removing Stop words
- Lower casing
- Tokenization
- Stemming
- Lemmatization

Preprocessing df_narr



In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english')) # removed to reduce noise.
lemmatizer = WordNetLemmatizer() #  reduce words to their base or dictionary form

def preprocess_text(text):
    text = text.lower() # lowercasing
    text = re.sub(r'http\S+|www\S+|https\S+', '', text) #remove urls
    text = text.translate(str.maketrans('', '', string.punctuation)) #remove punctuations
    text = re.sub(r'\d+', '', text) #remove numbers
    tokens = nltk.word_tokenize(text) #Breaks the cleaned text string into individual words (tokens).
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 2] #only keeps >2 and converts to their lemma form

    return ' '.join(tokens) #return clean text


In [ ]:
df_narr['cleaned_text'] = df_narr['consumer_complaint_narrative'].apply(preprocess_text)

New column cleaned_text in df_narr, which contains cleaned, lemmatized, stopword-free complaint narratives which is ready for sentiment analysis.  The above funtion takes messy complaint text and outputs a clean, lowercase, punctuation- and stopword-free, lemmatized string — ready for NLP analysis like sentiment modeling!

**Stemming in NLP?**
It is the process of reducing infected words to their stem. For instance, in figure 1, stemming with replace words “history” and “historical” with “histori”. Similarly, for the words finally and final.

Stemming is the process of removing the last few characters of a given word, to obtain a shorter form, even if that form doesn’t have any meaning in machine learning.

**Lemmatization in NLP?**
The purpose of lemmatization is same as that of stemming but overcomes the drawbacks of stemming. In stemming, for some words, it may not give may not give meaningful representation such as “Histori”. Here, lemmatization comes into picture as it gives meaningful word.

Lemmatization takes more time as compared to stemming because it finds meaningful word/ representation. Stemming just needs to get a base word and therefore takes less time.

Stemming has its application in Sentiment Analysis while Lemmatization has its application in Chatbots, human-answering.

**Tokenization**
Tokenization is the process of breaking a text into smaller units, usually words or subwords (called tokens).

## 4. Word cloud for different products

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
def wordcloud_for_product(df):
    products = df['product'].unique()
    print("Available products:\n")
    for i, prod in enumerate(products, 1):
        print(f"{i}. {prod}")

    choice = int(input("\nEnter the number corresponding to the product you want to see the word cloud for: "))
    selected_product = products[choice - 1]

    print(f"\nGenerating word cloud for: {selected_product}\n")

    text = ' '.join(df[df['product'] == selected_product]['cleaned_text'].dropna())

    wordcloud = WordCloud(
        width=400,
        height=400,
        background_color='white',
        stopwords=STOPWORDS,
        max_words=50,
        max_font_size=80,
        random_state=42
    ).generate(text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f"Word Cloud for Product: {selected_product}", fontsize=16)
    plt.axis('off')
    plt.show()

# Run this function with your dataframe containing 'cleaned_text'
wordcloud_for_product(df_narr)



**Why Word Clouds Often Disappoint in Financial NLP**
- Too many redacted placeholders like “xxx”, “xxxx”, or “name”.
- Generic words dominate, e.g. “account”, “bank”, “company” — words that appear in all complaints regardless of the issue.
- No differentiation by sentiment, so negative, positive, and neutral words are lumped together.
- Lack of context, like seeing the word “interest” but is it too high? too low? is unclear
- something visually cool but analytically shallow.

Better Alternatives - Instead of word clouds, consider - **Top Keywords by TF-IDF**
- gives distinguishing terms for a product or company that tells us what sets it apart from others.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def top_keywords_by_product(df, product_name, max_features=20):
    subset = df[df['product'] == product_name]

    if subset.empty:
        print(f"No data available for product: {product_name}")
        return

    tfidf = TfidfVectorizer(max_features=max_features, stop_words='english')
    X = tfidf.fit_transform(subset['cleaned_text'].dropna())
    keywords = tfidf.get_feature_names_out()

    print(f"\nTop {max_features} TF-IDF keywords for '{product_name}':")
    for word in keywords:
        print(f" - {word}")


In [ ]:
# View available product names
print(df_narr['product'].unique())

# Run the function with your choice
top_keywords_by_product(df_narr, "Credit card")


Highlights complaint themes: The TF-IDF keywords like fee, charge, late, statement, and payment clearly point to billing, payment issues, and account disputes, which are common in credit card complaints.

Extracts real language from consumers: Unlike predefined categories, these are consumer-driven signals, revealing what people emphasize in their own words.

Helps summarize the focus of complaints for each product. But here's what's less useful: Generic words like called, told, and especially xxxx, xxxxxxxx — these aren't meaningful.

These placeholders (e.g., xxxx) are usually masked sensitive data and shouldn't appear as key features.

##5.  Concise Abstractive Summary (via NLP Model):

|Goal    |	Method	                 |Tool                      |
| ------ | ------------------------- |------------------------- |
|Coherent| short human-like summary  |	Abstractive	Transformers (e.g. BART, T5)|
|Real    |consumer sentences	Extractive |	Sumy (LSA, LexRank)|

In [ ]:
from transformers import pipeline

# Step 1: Filter product
product = "Credit card"
subset = df_narr[df_narr['product'] == product]

# Step 2: Join all cleaned complaint narratives into one string
text_data = ' '.join(subset['cleaned_text'].dropna().tolist())

# Step 3: Use a summarization pipeline (e.g., distilbart or t5-small)
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Limit input size: summarizers can only handle ~1024-2048 tokens at a time
chunk = text_data[:2000]

# Step 4: Generate summary
summary = summarizer(chunk, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
print(f"\n📝 Summary of complaints for '{product}':\n{summary}")


In [3]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import warnings
warnings.filterwarnings('ignore')


csv_file_path = '/content/gdrive/My Drive/Colab Notebooks/consumer_complaints.csv'

class ProductBankSentiment:
    def __init__(self, csv_file_path):
        """Simple analyzer focused on product vs bank sentiment"""
        print("Loading data...")
        self.df = pd.read_csv(csv_file_path)
        print(f"Loaded {len(self.df):,} complaints")

        # Only keep complaints with narratives
        self.df = self.df[self.df['consumer_complaint_narrative'].notna()].copy()
        print(f"Found {len(self.df):,} complaints with narratives")

        # Process sentiment
        self.df['sentiment_score'] = self.df['consumer_complaint_narrative'].apply(self._analyze_sentiment)
        print("Sentiment analysis complete!")

    def _analyze_sentiment(self, text):
        """Simple sentiment analysis using TextBlob"""
        if pd.isna(text):
            return 0
        try:
            return TextBlob(str(text)).sentiment.polarity
        except:
            return 0

    def analyze_product(self, product_name):
        """
        Analyze sentiment for a specific product across all companies
        Returns top 3 positive and top 3 negative companies
        """
        # Filter data for the product
        product_data = self.df[self.df['product'] == product_name].copy()

        if len(product_data) == 0:
            print(f"No data found for product: {product_name}")
            return None

        # Group by company and calculate average sentiment
        company_sentiment = product_data.groupby('company').agg({
            'sentiment_score': ['mean', 'count'],
            'consumer_complaint_narrative': 'first'  # Get a sample narrative
        }).round(3)

        # Flatten column names
        company_sentiment.columns = ['avg_sentiment', 'complaint_count', 'sample_narrative']

        # Filter companies with at least 5 complaints (for reliability)
        company_sentiment = company_sentiment[company_sentiment['complaint_count'] >= 5]

        if len(company_sentiment) == 0:
            print(f"Not enough data for reliable analysis of {product_name}")
            return None

        # Sort by sentiment
        company_sentiment = company_sentiment.sort_values('avg_sentiment', ascending=False)

        # Get top 3 positive and negative
        top_positive = company_sentiment.head(3)
        top_negative = company_sentiment.tail(3)

        return {
            'product': product_name,
            'total_complaints': len(product_data),
            'companies_analyzed': len(company_sentiment),
            'top_positive': top_positive,
            'top_negative': top_negative,
            'product_data': product_data
        }

    def generate_narrative(self, results, sentiment_type='positive'):
        """Generate narrative for top companies"""
        if sentiment_type == 'positive':
            companies = results['top_positive']
            sentiment_word = "positive"
            description = "satisfaction and good experiences"
        else:
            companies = results['top_negative']
            sentiment_word = "negative"
            description = "frustration and poor experiences"

        if len(companies) == 0:
            return f"No companies with significant {sentiment_word} sentiment found."

        # Get top company details
        top_company = companies.index[0]
        avg_sentiment = companies.iloc[0]['avg_sentiment']
        complaint_count = companies.iloc[0]['complaint_count']

        narrative = f"""
For {results['product']}, consumers show the most {sentiment_word} sentiment toward {top_company}.

Key insights:
• Sentiment score: {avg_sentiment:.3f} (range: -1 to +1)
• Based on {complaint_count} complaints analyzed
• Customers frequently express {description} with their {results['product'].lower()} services
        """

        return narrative.strip()

    def print_analysis(self, product_name):
        """Print complete analysis for a product"""
        results = self.analyze_product(product_name)

        if not results:
            return

        print("="*80)
        print(f"SENTIMENT ANALYSIS: {product_name.upper()}")
        print("="*80)
        print(f"Total complaints analyzed: {results['total_complaints']:,}")
        print(f"Companies with sufficient data: {results['companies_analyzed']}")
        print()

        # TOP 3 POSITIVE COMPANIES
        print("🟢 TOP 3 RECOMMENDED COMPANIES (Most Positive Sentiment)")
        print("-"*60)
        positive_narrative = self.generate_narrative(results, 'positive')
        print(positive_narrative)
        print()

        print("Rankings:")
        for i, (company, row) in enumerate(results['top_positive'].iterrows(), 1):
            print(f"{i}. {company}")
            print(f"   Sentiment Score: {row['avg_sentiment']:.3f}")
            print(f"   Complaints Analyzed: {row['complaint_count']}")
            print()

        print("="*80)

        # TOP 3 NEGATIVE COMPANIES
        print("🔴 TOP 3 COMPANIES TO AVOID (Most Negative Sentiment)")
        print("-"*60)
        negative_narrative = self.generate_narrative(results, 'negative')
        print(negative_narrative)
        print()

        print("Rankings:")
        for i, (company, row) in enumerate(results['top_negative'].iterrows(), 1):
            print(f"{i}. {company}")
            print(f"   Sentiment Score: {row['avg_sentiment']:.3f}")
            print(f"   Complaints Analyzed: {row['complaint_count']}")
            print()

        print("="*80)

    def get_available_products(self):
        """Show available products for analysis"""
        product_counts = self.df['product'].value_counts()
        print("Available products for analysis:")
        print("-"*40)
        for product, count in product_counts.head(10).items():
            print(f"{product}: {count:,} complaints")
        return product_counts

# SIMPLE USAGE
if __name__ == "__main__":
    # Initialize analyzer
    csv_file_path = '/content/gdrive/My Drive/Colab Notebooks/consumer_complaints.csv'
    analyzer = ProductBankSentiment(csv_file_path)

    # Show available products
    analyzer.get_available_products()
    print()

    # Analyze specific products
    products_to_analyze = ['Credit card', 'Mortgage', 'Debt collection']  #  update prod to analyse

    for product in products_to_analyze:
        analyzer.print_analysis(product)
        print("\n" + "="*80 + "\n")

# QUICK SINGLE PRODUCT ANALYSIS
def quick_analysis(product_name):
    """Quick function to analyze just one product"""
    csv_file_path = '/content/gdrive/My Drive/Colab Notebooks/consumer_complaints.csv'
    analyzer = ProductBankSentiment(csv_file_path)
    analyzer.print_analysis(product_name)

# Usage examples:
# quick_analysis('Credit card')
# quick_analysis('Mortgage')

Loading data...
Loaded 555,957 complaints
Found 66,806 complaints with narratives
Sentiment analysis complete!
Available products for analysis:
----------------------------------------
Debt collection: 17,552 complaints
Mortgage: 14,919 complaints
Credit reporting: 12,526 complaints
Credit card: 7,929 complaints
Bank account or service: 5,711 complaints
Consumer Loan: 3,678 complaints
Student loan: 2,128 complaints
Prepaid card: 861 complaints
Payday loan: 726 complaints
Money transfers: 666 complaints

SENTIMENT ANALYSIS: CREDIT CARD
Total complaints analyzed: 7,929
Companies with sufficient data: 38

🟢 TOP 3 RECOMMENDED COMPANIES (Most Positive Sentiment)
------------------------------------------------------------
For Credit card, consumers show the most positive sentiment toward First Data Corporation.

Key insights:
• Sentiment score: 0.072 (range: -1 to +1)
• Based on 5 complaints analyzed
• Customers frequently express satisfaction and good experiences with their credit card ser